In [2]:
####     //      Import/Install packages/REST API       \\       ####
# Default packages
import time
import csv
import os
import json

# Preinstalled packages
import requests
import pandas as pd


# Define desired wd, where you want to save your .csv files
desired_wd = '/Users/jenskoning/Documents/Python_projects/dsr_media_nlp/dsr_media_nlp/Script/'
os.chdir(desired_wd)

# URL of our News API
base_url = 'https://api.newscatcherapi.com/v2/search'

# Your API key
X_API_KEY = 'uPGwd9h_DeqHKvxrDL5z9fu3t1Mc9rroBabtvX0iOuE'

In [8]:
####     //      Query Data       \\       ####
# Put your API key to headers in order to be authorized to perform a call
headers = {'x-api-key': X_API_KEY}

# Define your desired parameters
params = {
    'q': 'Digital Silk Road AND DSR',
    'lang': 'en',
    'to_rank': 10000,
    'page_size': 100,
    'from': '3 years ago', 
    'page': 1
    }

# Make a simple call with both headers and params
response = requests.get(base_url, headers=headers, params=params)

# Encode received results
results = json.loads(response.text.encode())
if response.status_code == 200:
    print('Done')
else:
    print(results)
    print('ERROR: API call failed.')


Done


In [10]:

####     //      Convert into pandas table       \\       ####
pandas_table = pd.DataFrame(results['articles'])
display(pandas_table)

,title,author,published_date,published_date_precision,link,clean_url,excerpt,summary,rights,rank,topic,country,language,authors,media,is_opinion,twitter_account,_score,_id
0,Assessing China's Digital Silk Road: A Transfo...,Amelia Cheatham,2020-12-18 15:38:29,full,https://www.cfr.org/blog/assessing-chinas-digi...,cfr.org,As part of China's massive Belt and Road Initi...,As part of China's massive Belt and Road Initi...,cfr.org,1830,news,US,en,"Amelia Cheatham,Claire Felter,Lindsay Maizland...",https://cdn.cfr.org/sites/default/files/styles...,False,@CFR_org,44.625183,89c5be6398de2cb01828fff0929544ee
1,China's Digital Silk Road Initiative: A Boon f...,None,2020-12-17 07:15:00,timezone unknown,https://thediplomat.com/2020/12/chinas-digital...,thediplomat.com,Some observers worry that it will equip author...,AdvertisementAs part of China's massive Belt a...,thediplomat.com,2310,news,ID,en,None,https://thediplomat.com/wp-content/uploads/202...,False,None,42.865353,81bcb48fb068dfc68ec4913c417baf95
2,China's Digital Silk Road and Africa's Technol...,Amelia Cheatham,2022-02-01 18:46:48,full,https://www.cfr.org/blog/chinas-digital-silk-r...,cfr.org,How will the DSR affect Africa's technological...,The Chinese government introduced the Digital ...,cfr.org,1830,news,US,en,"Amelia Cheatham,Claire Felter,Lindsay Maizland...",https://cdn.cfr.org/sites/default/files/styles...,False,@CFR_org,42.329834,7a1908a531a37020b7e8b7e92d060589
3,Connecting into the Social Sustainability Effe...,Bas Hooijmaaijers,2021-11-18 00:00:00,date,https://www.mdpi.com/2071-1050/13/22/12739,mdpi.com,There is a heated debate about the social-sust...,"Open AccessArticle by\n 1,* and 2,3 \n\n\n1\nM...",Creative Commons Attribution (CC-BY),449,science,CH,en,"[Jean-Marc F. Blanchard, Bas Hooijmaaijers]",https://www.mdpi.com/img/journals/sustainabili...,False,@MDPIOpenAccess,41.602710,01b69a81f6dea6b52ea10a7df5eca2c0
4,Challenge or Opportunity? How China's Digital ...,Ekaterina Blinova,2021-04-16 04:00:00,full,https://sputniknews.com/asia/20210416108264107...,sputniknews.com,Besides challenging the US' Big Tech dominance...,Besides challenging the US' Big Tech dominance...,"Sputnik News - World News, Breaking News & Top...",1079,news,RU,en,[],https://cdn1.img.sputniknews.com/images/07e4/0...,False,None,41.179974,0aaad30471a732327c4a48b3f3a3f24e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,The battle to modernize Japan,Jun Mukoyama,2021-01-05 08:00:00,full,https://www.japantimes.co.jp/opinion/2021/01/0...,japantimes.co.jp,In a country of fax machines and fragmented ce...,The creation of a digital agency is a top poli...,japantimes.co.jp,885,beauty,JP,en,[],https://cdn-japantimes.com/wp-content/uploads/...,True,None,19.291462,10a93644ba04420525afbb2760026ec4
65,Data Dominance: China Establishes Key EU Outpo...,Andreas Vou Andreas Vou View profile Follow,2022-01-26 14:59:06,full,https://www.theepochtimes.com/data-dominance-c...,theepochtimes.com,Republished with permission from BIRN. In Octo...,China's Huawei has come to dominate the 5G rol...,theepochtimes.com,1571,news,US,en,Andreas Vou,https://img.theepochtimes.com/assets/uploads/2...,False,@EpochTimes,16.188412,4814d7e6ece524e3c8db984dad04aa14
66,China-Russia cybersecurity ties taking darker ...,Swasti Rao,2023-03-10 08:44:38,full,https://theprint.in/opinion/china-russia-cyber...,theprint.in,Huawei is instrumental in strengthening the ba...,U\nbiquitous digital authoritarianism simmers ...,theprint.in,5567,news,IN,en,Swasti Rao,https://static.theprint.in/wp-content/uploads/...,True,@ThePrintIndia,15.871630,974f6ddcaba70c7c925786ed475e9175
67,"Data Dominance: In Cyprus, a Chinese Outpost i...",Andreas Vou,2021-12-07 07:29:57,full,https://balkaninsight.com/2021/12/07/data-domi...,balkaninsight.com,China's Huawei has come to dominate the 5G rol...,China's Huawei has come to dominate the 5G rol...,balkaninsight.com,8628,news,RS,en,[Andreas Vou],https://balkaninsight.com/wp-content/upl

In [11]:
# Variable to store all found news articles
all_news_articles = []

# Ensure that we start from page 1
params['page'] = 1

# Infinite loop which ends when all articles are extracted
while True:

    # Wait for 1 second between each call
    time.sleep(1)

    # GET Call from previous section enriched with some logs
    response = requests.get(base_url, headers=headers, params=params)
    results = json.loads(response.text.encode())
    if response.status_code == 200:
        print(f'Done for page number => {params["page"]}')


        # Adding your parameters to each result to be able to explore afterwards
        for i in results['articles']:
            i['used_params'] = str(params)


        # Storing all found articles
        all_news_articles.extend(results['articles'])

        # Ensuring to cover all pages by incrementing "page" value at each iteration
        params['page'] += 1
        if params['page'] > results['total_pages']:
            print("All articles have been extracted")
            break
        else:
            print(f'Proceed extracting page number => {params["page"]}')
    else:
        print(results)
        print(f'ERROR: API call failed for page number => {params["page"]}')
        break

print(f'Number of extracted articles => {str(len(all_news_articles))}')

Done for page number => 1
All articles have been extracted
Number of extracted articles => 69


In [12]:
# Generate CSV from Pandas table
# Create Pandas table
pandas_table = pd.DataFrame(all_news_articles)

# Generate CSV
pandas_table.to_csv('extracted_news_articles.csv', encoding='utf-8', sep=';')